In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import re #regex

Eventually create a list of major codes and run a for loop to get all dataframes

In [ ]:
#From https://stackoverflow.com/questions/13332192/inherent-way-to-save-web-page-source
# This puts all the course catalog webpages into a folder 'sites'
depts = ['ANTH','BIOI','BIOL','CHEM','CHIN','CLAS','CCS','COGS','COMM','CGS','CAT','DSC','DSGN','DOC','ECON','EDS','ENG','BENG','CSE','ECE','MAE','NANO','SE','ENVR','ESYS','ETHN','FMPH','FILM','GLBH','HIST','HDS','HR','HUM','INTL','JAPN','JWSP','LATI','LAWS','LING','LIT','MGT','MATH','MUS','PHIL','PHYS','POLI','PSYC','RELI','SCIS','SIO','SOC','THEA','TWS','USP','VIS']

for dept in depts:
    url =  'https://www.ucsd.edu/catalog/courses/' + dept + '.html'
    page = urllib.request.urlopen(url)
    page_content = page.read()
    with open('sites/' + dept + '_courses.html', 'wb') as fid:
        fid.write(page_content)

In [ ]:
#this beautifulsoups those catalogs from the folder
def extract_course_catalog(dept): #where dept is a string
    f = open('/sites' + dept + '_courses.html') #dept is all caps     
    soup = BeautifulSoup(f, 'html.parser')

In [2]:
dept = 'MATH'
site_url = 'https://www.ucsd.edu/catalog/courses/MATH.html'
page = requests.get(site_url)
soup = BeautifulSoup(page.content, 'html.parser')

#https://stackoverflow.com/questions/49010813/how-to-find-a-particular-class-in-beautifulsoup-using-findall

# Looking at just names and descriptions w/ htmlclass separation

In [3]:
#full_list = soup.find_all("p", {"class": "anchor-parent"}) 

names = soup.find_all("p", {"class": "course-name"})
descriptions = soup.find_all("p", {"class": "course-descriptions"}) #includes prereqs (so want to extract that)

In [7]:
name_list = [] #this is in HTML contains all course header info

for x in names:
    name_list.append(str(x))

In [8]:
course_ids = []
course_names = []
course_units = [] 

for n in np.arange(len(name_list) - 1):
    #for i in np.arange(len(name_list[n]) - 1):
    x = name_list[n]
    course_ids.append(x[len('<p class="course-name">') + len(dept) + 1 : x.index('.')])
    course_names.append(x[x.index('.')+2:len(x)-8])
    course_units.append(x[len(x)-6])

## Some course names are weird (why?) e.g. 'Foundations\n\t\t  of Teaching and Learning Mathematics I'

## Course units cut off (why?)

In [9]:
course_nums = []

for i in course_ids:
    course_nums.append(re.findall(r'\d+|\D+', i)) #regex makes id's in format 'Num' + 'Letter'    

#regex for this \d+|\D+

# just nums: r'^\d+'

#### use list comprehension it is cleaner

finding indices of class category

In [10]:
for i in course_nums:
    if len(i[0]) == 2:
        try:
            lower_div_i = course_ids.index(i[0]+i[1]) #last index of lower divisions
        except IndexError:
            lower_div_i = course_ids.index(i[0])
    if int(i[0]) < 200:
        try:
            upper_div_i = course_ids.index(i[0]+i[1]) #" " of upper divisions
        except IndexError:
            upper_div_i = course_ids.index(i[0])
        
#[i for i,val in enumerate(l) if val==]

https://stackoverflow.com/questions/6294179/how-to-find-all-occurrences-of-an-element-in-a-list


### How to find last value before 200

In [11]:
#converting description blocks into lists

desc_list = []
for i in descriptions:
    desc_list.append(re.findall(r"\w+,*", str(i)))

In [12]:
# Removing graduate courses

course_ids = course_ids[0:upper_div_i + 1]
course_names = course_names[0:upper_div_i + 1]
course_units = course_units[0:upper_div_i + 1]
desc_list = desc_list[0:upper_div_i + 1]

In [13]:
#extracting prerequisites.  

prereq_list = []

for i in np.arange(len(desc_list)):
    x = desc_list[i]
    try: 
        prereq_list.append(x[x.index('Prerequisites')+2:len(x)-1])
    except ValueError:
        print('prereq extraction error at index', i) 
        prereq_list.append('None')

## Check where the ValueErrors occur!  In some cases it may be faulty html e.g Math 157 has weird indent and misses prereq section

Before cell below, need to clear the following words/strings: 
'either'

and turn this:

'Students',
  'who',
  'have',
  'not',
  'completed',
  'listed',
  'prerequisites',
  'may',
  'enroll',
  'with',
  'consent',
  'of',
  'instructor'
  
into just " or + consent of instructor"


### might have to parse for or and make the lists 3d

In [15]:
prereqs_2d = []
for n in np.arange(len(course_ids)):
    ps = prereq_list[n]
    ps_2d = [[]]
    inner_i = 0
    for i in np.arange(len(ps)):    
        if ps[i] != 'and':
            ps_2d[inner_i].append(ps[i])
        #if ps[i] == 'students' or 
        else:
            ps_2d.append([])
            inner_i = inner_i + 1
    prereqs_2d.append(ps_2d)

In [ ]:
n = [['MATH', '140A', 'or', 'consent', 'of', 'instructor'], [asdasdasd]]

prereqs_3d = []
for n in np.arange(len(course_ids)):
    ps = prereq_list[n]:
        for i in np.arange(len(ps)):
            for i2 in ps[i]:
                if i2 

In [32]:
course_ids[36]

'112A'

In [19]:
prereqs_2d[36]

[['strong',
  'MATH',
  '11',
  'or',
  'MATH',
  '180A',
  'or',
  'MATH',
  '183',
  'or',
  'MATH',
  '186,'],
 ['MATH', '18', 'or', 'MATH', '31AH,'],
 ['MATH', '20D,'],
 ['BILD',
  '1',
  'Students',
  'who',
  'have',
  'not',
  'completed',
  'listed',
  'prerequisites',
  'may',
  'enroll',
  'with',
  'consent',
  'of',
  'instructor']]

all classes are going to have to be in the same object unless more looping

In [23]:
for prereq in prereqs_for_some_class:
    preprereqs = [] # prereqs for this prereq 
    look at prereqs of the prereq:
        repeat?


for i in prereqs:
    prereqs_3d[course_ids.index(i)]
    

SyntaxError: invalid syntax (<ipython-input-23-889b7b9f37a0>, line 3)

# Want to create dataframe with Columns: name, ID, prereqs, descs (?)

In [ ]:
course_info_df = pd.DataFrame({'Course id': course_ids, 'Course name': course_names, 'Course units' = course_units, 'Prereqs' = })

Course names array must be in 2d.

[[Math 2, Introduction to College Mathematics], [MATH 3, Precalculus] ...] 

In [78]:

#course_page = course_page[0:int(np.where(course_page == '<p class="course-name">MATH 200A-B-C. Algebra (4-4-4)</p>')[0])]

(205,)


In [51]:
courses = []
start = ' <p class="course-name"> '
stop = '</p>'
for i in course_page:
    for s in i:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-51-89e5bcb3f799>, line 5)

# GRAPHS

adjacency matrix or array of vertex pairs?

-adj lists

have to consider inter departmental case and how to access their respective course_ids etc

The loop should keep going as long as it can find the class in a prereq object.

dictionaries?  [Dept, ID] : [prereqs]

In [ ]:
def get_prereq(dept, num):
    a = course_ids.index(num) # [!] need to specify for dept
    return prereqs_3d[a]

#### The recursive part

input: course
intermediary input: the course's immediate prerequisites

the loop: should iterate down until no more prereq

output: A graph

In [ ]:
eg_prq = [ [['CSE', '8B'], ['CSE','11']],  [['MATH', '18'], ['MATH', '31AH']], [['MATH', '20E']], [['MATH','180A']] ]

map_graph = []
for i1 in eg_prq:
    for i2 in i1:
        prqs = get_prereq(i2[0], i2[1])
        
        
        
    map_graph.append([c1, c2])
        
        #dept = n[0][0]
                
                if eg_prq[i] == prereq_list[x] 

In [9]:
a = []
a.append(1)


In [10]:
a

[1]

In [ ]:
{'MATH 2' : ([name], [prereqs], [], [])}

In [34]:
name_list

['<p class="course-name">MATH 2. Introduction to College Mathematics (4)</p>',
 '<p class="course-name">MATH 3C. Precalculus (4)</p>',
 '<p class="course-name">MATH 4C. Precalculus for Science and Engineering (4)</p>',
 '<p class="course-name">MATH 10A. Calculus I (4)</p>',
 '<p class="course-name">MATH 10B. Calculus II (4)</p>',
 '<p class="course-name">MATH 10C. Calculus III (4)</p>',
 '<p class="course-name">MATH 11. Calculus-Based Introductory Probability and Statistics (5)</p>',
 '<p class="course-name">MATH 15A. Introduction to Discrete Mathematics (4)</p>',
 '<p class="course-name">MATH 18. Linear Algebra (4)</p>',
 '<p class="course-name">MATH 20A. Calculus for Science and Engineering (4)</p>',
 '<p class="course-name">MATH 20B. Calculus for Science and Engineering (4)</p>',
 '<p class="course-name">MATH 20C. Calculus\n\t\t  and Analytic Geometry for Science and Engineering (4)</p>',
 '<p class="course-name">MATH 20D. Introduction to Differential Equations (4)</p>',
 '<p class=

create a tuple with all values instantly

In [ ]:
class course_node(name):
    def __init__(self, name):
        self.name = name
        self.prereqs = []

In [ ]:
def make_course_node():
    #feeds cleaned data into it
    

In [ ]:
def make_graph()

